# Preprocessing of the training data

In [5]:
path_training_dest = "/content/drive/MyDrive/Projet Twitter"
train_path = "/content/drive/MyDrive/Projet Twitter/challenge_data/train_tweets"

path_test_dest = "/content/drive/MyDrive/Projet Twitter"
test_path = "/content/drive/MyDrive/Projet Twitter/challenge_data/eval_tweets"

If need for mounting a Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import seaborn as sb
import openpyxl
import plotly.express as px
import plotly.graph_objects as go
import torch
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import os
import re
import gensim.downloader as api
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

!pip install emoji
import emoji

!pip install transformers torch

import tensorflow_hub as hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 20.4 MB/s eta 0:00:00


In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

Useful functions

In [9]:
def count_emojis(text):
    return sum(1 for char in text if char in emoji.EMOJI_DATA)

In [10]:
def count_excessive_punctuation(text):
    # Regex pour détecter des séquences de ponctuations (deux ou plus)
    matches = re.findall(r"[!?.,]{2,}", text)
    return len(matches)

In [11]:
def ratio_uppercase(text):
    # Filtrer les lettres majuscules et totales
    total_letters = sum(1 for char in text if char.isalpha())
    uppercase_letters = sum(1 for char in text if char.isupper())
    # Éviter une division par zéro
    return uppercase_letters / total_letters if total_letters > 0 else 0

In [12]:
def generate_use_embedding(text):
    # Appliquer l'encodeur et convertir le résultat en array numpy
    embedding = embed([text]).numpy()
    # Calculer la moyenne de l'embedding pour obtenir un vecteur moyen
    return embedding.mean(axis=0)

# Agréger les vecteurs par période (par exemple, moyenne)
def aggregate_vectors(vectors):
  return np.mean(np.stack(vectors), axis=0)

In [13]:
def score_emotion(tweet):
    # Appliquer le modèle de scoring
    encoder_input = tokenizer(tweet, return_tensors='pt')
    output = model(**encoder_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    emotion_intensity = scores[0] + scores[2]
    return emotion_intensity

In [14]:
spam_keywords = [ # Mots-clés liés aux concours et cadeaux
                 "Free", "Giveaway", "Prize", "Gift", "Reward", "Contest", "Sweepstake", "Enter to win", "Claim your", "Exclusive", "Redeem", "Limited time", "Lucky draw", "Grand prize", "Bonus",
                  # Expressions fréquentes dans les spams
                  "Act now", "Don't miss", "Limited offer", "Hurry up", "Click here", "Sign up", "Subscribe", "Buy now", "Special discount", "Exclusive deal", "Limited stock", "Shop now", "Hot deal", "Lowest price", "Flash sale", "Only today", "Promo code", "Coupon", "Sale ends soon", "100% free", "Free trial", "No purchase necessary", "Discount available", "Best price", "Cheapest offer", "Earn money", "Extra cash", "Make money", "Easy money", "Work from home", "Passive income", "Referral", "Affiliate", "Sponsor",
                  # Expressions pour générer du clic ou inciter à agir
                  "Follow us", "Tag your friends", "Retweet", "Share to win", "DM us", "Message us", "Check this out", "Learn more", "Limited access", "Offer valid until", "Don't miss out", "Get it now", "Click the link", "Sign up now", "Swipe up", "Order now", "Instant access", "Join us", "Become a member",
                  # Autres termes indiquant des spams de faux comptes ou bots
                  "Bitcoin", "Crypto", "NFT", "Investment", "Forex", "Binary options", "Trading", "Double your money", "Crypto giveaway", "ETH", "BTC", "Ethereum", "Token", "Blockchain", "Profit guaranteed", "Investment opportunity", "Low risk", "High return", "Secure your spot", "Be your own boss",
                  # Faux messages d'alerte ou de sécurité
                  "Urgent", "Attention", "Your account", "Suspended", "Verification required", "Click to verify", "Account locked", "Secure your account", "Login now", "Change your password", "Phishing", "Hacked", "Unauthorized access",
                  # Faux messages ou arnaques
                  "You've won", "Congratulations", "You have been selected", "Claim now", "Exclusive offer", "Act fast", "Get started", "First 100 people", "Lucky winner", "Cash prize", "Biggest sale", "Once in a lifetime", "Giveaway alert", "Official", "Verified", "Best deal" ]

def remove_tweets_by_keywords(df, keywords):
    # Création d'un masque pour les tweets qui contiennent au moins un mot-clé
    mask = df['Tweet'].apply(lambda tweet: any(keyword.lower() in tweet.lower() for keyword in keywords))
    # Appliquer le masque et retourner la dataframe filtrée
    # enlever les retweets
    df = df[~df['Tweet'].str.startswith('RT')]

    # enlever les tweets avec un URL
    df = df[~df['Tweet'].str.contains(r'http[s]?://', na=False)]
    return df[~mask]

In [15]:
def remove_mentions(tweet):
    # Diviser le tweet en mots et filtrer ceux qui commencent par '@'
    words = tweet.split()
    words = [word for word in words if not word.startswith('@')]
    # Joindre les mots filtrés pour reformer le tweet
    return ' '.join(words)

Preprocessing of every match

In [ ]:
dataframes = []
for fichier in os.listdir(train_path):
    if fichier.endswith('.csv'):
      chemin_complet = os.path.join(train_path, fichier)
      # Chargement des données
      df = pd.read_csv(chemin_complet)

      # Création des labels
      labels = df.groupby('ID')['EventType'].sum().reset_index()
      labels['EventType'] = labels['EventType'].apply(lambda x: 1 if x > 0 else 0)

      # Filtrages des tweets inutiles
      df = df[~df['Tweet'].str.startswith('RT')]
      df = df[~df['Tweet'].str.contains(r'http[s]?://', na=False)]

      df = remove_tweets_by_keywords(df, spam_keywords)
      df['Tweet'] = df['Tweet'].apply(remove_mentions)

      # Gestion des PeriodID
      period_max = df['PeriodID'].max()
      df_by_period = pd.DataFrame()
      df_by_period['PeriodID'] = np.arange(0, period_max + 1, 1)
      df_by_period['ID'] = labels['ID']
      df_by_period = df_by_period.merge(labels, on='ID', how='left')

      # Nombre de tweets et taux d'accroissement
      tweets_per_period = df.groupby('PeriodID').size().reset_index(name='TweetCount')
      tweets_per_period['GrowthRate'] = tweets_per_period['TweetCount'].pct_change() * 100
      df_by_period = df_by_period.merge(tweets_per_period, on='PeriodID', how='left')

      nb_total_tweets = tweets_per_period['TweetCount'].sum()
      df_by_period['nb_tweets_normalize'] = tweets_per_period['TweetCount'] / nb_total_tweets
      df_by_period['rate_increase'] = tweets_per_period['GrowthRate']
      df_by_period.loc[0, 'rate_increase'] = 0

      # Comptage des emojis
      df['EmojiCount'] = df['Tweet'].apply(count_emojis)
      emoji_count_per_period = df.groupby('PeriodID')['EmojiCount'].sum().reset_index()
      emoji_count_per_period['TotalEmojiCount'] = emoji_count_per_period['EmojiCount'] / nb_total_tweets
      df_by_period = df_by_period.merge(emoji_count_per_period[['PeriodID', 'TotalEmojiCount']], on='PeriodID', how='left')

      # Comptage de la ponctuation excessive
      df['ExcessivePunctCount'] = df['Tweet'].apply(count_excessive_punctuation)
      punctuation_count_per_period = df.groupby('PeriodID')['ExcessivePunctCount'].sum().reset_index()
      punctuation_count_per_period['TotalExcessivePunctCount'] = punctuation_count_per_period['ExcessivePunctCount'] / nb_total_tweets
      df_by_period = df_by_period.merge(punctuation_count_per_period[['PeriodID', 'TotalExcessivePunctCount']], on='PeriodID', how='left')

      # Ratio de majuscules
      df['UppercaseRatio'] = df['Tweet'].apply(ratio_uppercase)
      avg_uppercase_ratio = df.groupby('PeriodID')['UppercaseRatio'].mean().reset_index()
      df_by_period = df_by_period.merge(avg_uppercase_ratio.rename(columns={'UppercaseRatio': 'AvgUppercaseRatio'}), on='PeriodID', how='left')

      # Vectorisation
      # Calcul des embeddings USE pour chaque tweet
      df['Embedding'] = df['Tweet'].apply(generate_use_embedding)

      # Obtenir la taille de l'embedding
      embedding_dim = 512  # Taille du vecteur USE

      # Créer des colonnes pour chaque dimension d'embedding
      embedding_cols = [f'USE_Dim_{i}' for i in range(embedding_dim)]

      # Ajouter ces colonnes au dataframe df
      df[embedding_cols] = pd.DataFrame(df['Embedding'].tolist(), index=df.index)

      # Regrouper les embeddings par ID et calculer la moyenne
      df_grouped_by_id = df.groupby('ID')[embedding_cols].mean().reset_index()

      # Fusionner les embeddings agrégés par ID avec df_by_period sur 'ID'
      df_by_period = df_by_period.merge(df_grouped_by_id, on='ID', how='left')

      dataframes.append(df_by_period)

df_by_period = pd.concat(dataframes)
df_by_period.fillna(0, inplace=True)
df_by_period.to_csv(path_training_dest+'/df_by_period_nuit.csv', index=False)

<ipython-input-13-092602379e7a>:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[embedding_cols] = pd.DataFrame(df['Embedding'].tolist(), index=df.index)
<ipython-input-13-092602379e7a>:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[embedding_cols] = pd.DataFrame(df['Embedding'].tolist(), index=df.index)
<ipython-input-13-092602379e7a>:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at onc

We add the feature of average tweet length by period

In [16]:
df_by_period = pd.read_csv(path_training_dest+'/df_train_nuit_finale.csv')
# Nouveau dataframe pour stocker avg_tweet_len par ID
avg_tweet_len_by_id = []

for fichier in os.listdir(train_path):
    if fichier.endswith('.csv'):
        chemin_complet = os.path.join(train_path, fichier)

        # Chargement des données
        df = pd.read_csv(chemin_complet)
        # Filtrages des tweets inutiles
        df = df[~df['Tweet'].str.startswith('RT')]
        df = df[~df['Tweet'].str.contains(r'http[s]?://', na=False)]

        df = remove_tweets_by_keywords(df, spam_keywords)
        df['Tweet'] = df['Tweet'].apply(remove_mentions)

        # Calcul de la longueur moyenne des tweets par ID
        df['TweetLength'] = df['Tweet'].apply(len)
        avg_tweet_len = df.groupby('ID')['TweetLength'].mean().reset_index()
        avg_tweet_len.rename(columns={'TweetLength': 'AvgTweetLength'}, inplace=True)
        avg_tweet_len_by_id.append(avg_tweet_len)

# Combiner les données de avg_tweet_len pour tous les fichiers
avg_tweet_len_df = pd.concat(avg_tweet_len_by_id)
avg_tweet_len_df = avg_tweet_len_df.groupby('ID', as_index=False).mean()
df_by_period = df_by_period.merge(avg_tweet_len_df, on='ID', how='left')
df_by_period.to_csv(path_training_dest+'/df_by_period_nuit_tweets_len.csv', index=False)

# Preprocessing of the test data

In [ ]:
dataframes_test = []

for fichier in os.listdir(test_path):
    if fichier.endswith('.csv'):
        chemin_complet = os.path.join(test_path, fichier)
        df = pd.read_csv(chemin_complet)
        period_max = np.max(df["PeriodID"].tolist())
        df_by_period = pd.DataFrame()
        df_by_period["PeriodID"] = np.arange(0,period_max+1,1)

        # Filtrages des tweets inutiles
        df = df[~df['Tweet'].str.startswith('RT')]
        df = df[~df['Tweet'].str.contains(r'http[s]?://', na=False)]

        df = remove_tweets_by_keywords(df, spam_keywords)
        df['Tweet'] = df['Tweet'].apply(remove_mentions)

        # Gestion des PeriodID
        period_max = df['PeriodID'].max()
        df_by_period = pd.DataFrame()
        df_by_period['PeriodID'] = np.arange(0, period_max + 1, 1)
        df_by_period['ID'] = labels['ID']
        df_by_period = df_by_period.merge(labels, on='ID', how='left')

        # Nombre de tweets et taux d'accroissement
        tweets_per_period = df.groupby('PeriodID').size().reset_index(name='TweetCount')
        tweets_per_period['GrowthRate'] = tweets_per_period['TweetCount'].pct_change() * 100
        df_by_period = df_by_period.merge(tweets_per_period, on='PeriodID', how='left')

        nb_total_tweets = tweets_per_period['TweetCount'].sum()
        df_by_period['nb_tweets_normalize'] = tweets_per_period['TweetCount'] / nb_total_tweets
        df_by_period['rate_increase'] = tweets_per_period['GrowthRate']
        df_by_period.loc[0, 'rate_increase'] = 0

        # Comptage des emojis
        df['EmojiCount'] = df['Tweet'].apply(count_emojis)
        emoji_count_per_period = df.groupby('PeriodID')['EmojiCount'].sum().reset_index()
        emoji_count_per_period['TotalEmojiCount'] = emoji_count_per_period['EmojiCount'] / nb_total_tweets
        df_by_period = df_by_period.merge(emoji_count_per_period[['PeriodID', 'TotalEmojiCount']], on='PeriodID', how='left')

        # Comptage de la ponctuation excessive
        df['ExcessivePunctCount'] = df['Tweet'].apply(count_excessive_punctuation)
        punctuation_count_per_period = df.groupby('PeriodID')['ExcessivePunctCount'].sum().reset_index()
        punctuation_count_per_period['TotalExcessivePunctCount'] = punctuation_count_per_period['ExcessivePunctCount'] / nb_total_tweets
        df_by_period = df_by_period.merge(punctuation_count_per_period[['PeriodID', 'TotalExcessivePunctCount']], on='PeriodID', how='left')

        # Ratio de majuscules
        df['UppercaseRatio'] = df['Tweet'].apply(ratio_uppercase)
        avg_uppercase_ratio = df.groupby('PeriodID')['UppercaseRatio'].mean().reset_index()
        df_by_period = df_by_period.merge(avg_uppercase_ratio.rename(columns={'UppercaseRatio': 'AvgUppercaseRatio'}), on='PeriodID', how='left')

        # Vectorisation
        # Calcul des embeddings USE pour chaque tweet
        df['Embedding'] = df['Tweet'].apply(generate_use_embedding)

        # Obtenir la taille de l'embedding
        embedding_dim = 512  # Taille du vecteur USE

        # Créer des colonnes pour chaque dimension d'embedding
        embedding_cols = [f'USE_Dim_{i}' for i in range(embedding_dim)]

        # Ajouter ces colonnes au dataframe df
        df[embedding_cols] = pd.DataFrame(df['Embedding'].tolist(), index=df.index)

        # Regrouper les embeddings par ID et calculer la moyenne
        df_grouped_by_id = df.groupby('ID')[embedding_cols].mean().reset_index()

        # Fusionner les embeddings agrégés par ID avec df_by_period sur 'ID'
        df_by_period = df_by_period.merge(df_grouped_by_id, on='ID', how='left')

        dataframes.append(df_by_period)

df_by_period = pd.concat(dataframes)
df_by_period.fillna(0, inplace=True)
df_by_period.to_csv(path_test_dest+"/df_test_nuit.csv", index=False)

In [18]:
df_by_period = pd.read_csv(path_test_dest+'/df_test_nuit.csv')
# Nouveau dataframe pour stocker avg_tweet_len par ID
avg_tweet_len_by_id = []

for fichier in os.listdir(test_path):
    if fichier.endswith('.csv'):
        chemin_complet = os.path.join(test_path, fichier)

        # Chargement des données
        df = pd.read_csv(chemin_complet)
        # Filtrages des tweets inutiles
        df = df[~df['Tweet'].str.startswith('RT')]
        df = df[~df['Tweet'].str.contains(r'http[s]?://', na=False)]

        df = remove_tweets_by_keywords(df, spam_keywords)
        df['Tweet'] = df['Tweet'].apply(remove_mentions)

        # Calcul de la longueur moyenne des tweets par ID
        df['TweetLength'] = df['Tweet'].apply(len)
        avg_tweet_len = df.groupby('ID')['TweetLength'].mean().reset_index()
        avg_tweet_len.rename(columns={'TweetLength': 'AvgTweetLength'}, inplace=True)
        avg_tweet_len_by_id.append(avg_tweet_len)

# Combiner les données de avg_tweet_len pour tous les fichiers
avg_tweet_len_df = pd.concat(avg_tweet_len_by_id)
avg_tweet_len_df = avg_tweet_len_df.groupby('ID', as_index=False).mean()
df_by_period = df_by_period.merge(avg_tweet_len_df, on='ID', how='left')
df_by_period.to_csv(path_training_dest+'/df_test_nuit_tweets_len.csv', index=False)